In [9]:
import pandas as pd

# Load historical hourly prices (same CSV from Example 1)
df = pd.read_csv("data/omie_day_ahead_prices_es_history.csv")

df["date"] = pd.to_datetime(df["date"])
df.head()

,date,hour,price_eur_mwh
0,2024-10-01,1,104.00
1,2024-10-01,2,97.68
2,2024-10-01,3,97.68
3,2024-10-01,4,94.86
4,2024-10-01,5,84.01


In [11]:
# Aggregate to daily metrics used for monitoring

daily = (
    df
    .groupby("date")
    .agg(
        daily_avg_price=("price_eur_mwh", "mean"),
        daily_max_price=("price_eur_mwh", "max"),
        daily_min_price=("price_eur_mwh", "min"),
        daily_std=("price_eur_mwh", "std"),
    )
    .reset_index()
)

daily = daily.sort_values("date").reset_index(drop=True)
daily.tail()

,date,daily_avg_price,daily_max_price,daily_min_price,daily_std
68,2024-12-09,67.188750,137.21,3.85,51.993393
69,2024-12-10,134.224167,181.00,100.70,23.861920
70,2024-12-11,140.609167,179.07,113.05,22.068276
71,2024-12-12,146.672917,177.38,112.60,20.480461
72,2024-12-13,145.290417,172.35,114.95,17.799757


In [13]:
# Rolling references for monitoring
ROLLING_WINDOW = 14

daily["rolling_avg_price"] = daily["daily_avg_price"].rolling(ROLLING_WINDOW).mean()
daily["rolling_std"] = daily["daily_avg_price"].rolling(ROLLING_WINDOW).std()

daily.tail()

,date,daily_avg_price,daily_max_price,daily_min_price,daily_std,rolling_avg_price,rolling_std
68,2024-12-09,67.188750,137.21,3.85,51.993393,113.421190,40.354119
69,2024-12-10,134.224167,181.00,100.70,23.861920,112.776815,39.910843
70,2024-12-11,140.609167,179.07,113.05,22.068276,113.073631,40.115400
71,2024-12-12,146.672917,177.38,112.60,20.480461,114.108214,40.826772
72,2024-12-13,145.290417,172.35,114.95,17.799757,115.393304,41.550652


In [19]:
# Alert thresholds (adjusted to reduce noise)
PRICE_SPIKE_THRESHOLD = 0.30    # +30% vs recent average
VOLATILITY_THRESHOLD = 40.0     # €/MWh
PEAK_PRICE_THRESHOLD = 170.0    # €/MWh

alerts = []

for _, row in daily.iterrows():
    reasons = []

    # Compute spike trigger level (only if rolling avg exists)
    if pd.notna(row["rolling_avg_price"]):
        spike_trigger_level = row["rolling_avg_price"] * (1 + PRICE_SPIKE_THRESHOLD)

        if row["daily_avg_price"] > spike_trigger_level:
            reasons.append("Price spike vs recent average")
    else:
        spike_trigger_level = None

    if row["daily_std"] > VOLATILITY_THRESHOLD:
        reasons.append("High intraday volatility")

    if row["daily_max_price"] > PEAK_PRICE_THRESHOLD:
        reasons.append("Extreme hourly peak")

    if reasons:
        alerts.append({
            "date": row["date"],
            "daily_avg_price": round(row["daily_avg_price"], 2),
            "rolling_avg_price": round(row["rolling_avg_price"], 2) if pd.notna(row["rolling_avg_price"]) else None,
            "price_spike_trigger_level": round(spike_trigger_level, 2) if spike_trigger_level is not None else None,
            "daily_std": round(row["daily_std"], 2),
            "daily_max_price": round(row["daily_max_price"], 2),
            "alert_reason": ", ".join(reasons)
        })

alerts_df = pd.DataFrame(alerts)
alerts_df


,date,daily_avg_price,rolling_avg_price,price_spike_trigger_level,daily_std,daily_max_price,alert_reason
0,2024-10-01,98.07,NaN,NaN,32.32,173.81,Extreme hourly peak
1,2024-10-10,76.18,NaN,NaN,42.69,180.00,"High intraday volatility, Extreme hourly peak"
2,2024-10-14,104.17,67.30,87.49,28.77,178.61,"Price spike vs recent average, Extreme hourly ..."
3,2024-10-15,87.23,66.52,86.48,15.80,125.89,Price spike vs recent average
4,2024-10-21,94.00,63.74,82.86,29.48,181.00,"Price spike vs recent average, Extreme hourly ..."
5,2024-10-31,100.77,71.12,92.46,23.24,148.72,Price spike vs recent average
6,2024-11-04,108.72,81.04,105.36,20.74,160.54,Price spike vs recent average
7,2024-11-05,117.05,82.69,107.50,27.20,193.00,"Price spike vs recent average, Extreme hourly ..."
8,2024-11-06,116.83,85.31,110.90,21.29,171.89,"Price spike vs recent average, Extreme hourly ..."
9,2024-11-12,64.74,96.40,125.31,44.90,128.86,High intraday volatility


In [21]:
# Focus on the most recent alerts
alerts_df.tail(5)

,date,daily_avg_price,rolling_avg_price,price_spike_trigger_level,daily_std,daily_max_price,alert_reason
15,2024-12-09,67.19,113.42,147.45,51.99,137.21,High intraday volatility
16,2024-12-10,134.22,112.78,146.61,23.86,181.00,Extreme hourly peak
17,2024-12-11,140.61,113.07,147.00,22.07,179.07,Extreme hourly peak
18,2024-12-12,146.67,114.11,148.34,20.48,177.38,Extreme hourly peak
19,2024-12-13,145.29,115.39,150.01,17.80,172.35,Extreme hourly peak


## Ops Task Automator: Price Alerts

This notebook adds an operational layer on top of the OMIE day-ahead price data, turning historical analysis into actionable monitoring.

The objective is simple: **identify days that are materially different from recent market conditions and deserve attention**.

---

### What the automation monitors
Daily prices are summarised into three operational signals:

- **Price level**: how expensive the day is compared to recent history  
- **Intraday volatility**: how unstable prices are within the day  
- **Extreme peaks**: whether a small number of hours drive disproportionate cost

These dimensions capture most situations where market conditions become operationally relevant.

---

### Alert rules and thresholds
The alert thresholds are calibrated to balance sensitivity and noise:

- **Price spike (+30% vs 14-day average)**  
  Flags days where the overall price level is materially higher than the recent baseline.

- **High intraday volatility (> 40 €/MWh)**  
  Identifies unstable days where hourly prices vary significantly, increasing uncertainty and risk.

- **Extreme hourly peak (> 180 €/MWh)**  
  Focuses on genuinely extreme price spikes, reducing alert fatigue while highlighting hours that can drive outsized costs.

A 14-day rolling window is used to reflect recent market conditions while remaining responsive to changes.

---

### Business interpretation
- Price spike alerts highlight potential shifts in market conditions that may impact planning assumptions.
- Volatility alerts flag days where forecasting and operational risk are elevated.
- Extreme peak alerts surface hours where flexibility or demand management would have the greatest value.

---

### Output
The result is a concise table of flagged days with clear reasons, suitable for daily or weekly operational review.

The goal is not to automate decisions, but to **ensure that unusual and high-risk market conditions are surfaced early and consistently**.